# 🗂️ SQLite SQL — A Comprehensive Guide for Learners and Developers

*A Deep Dive into the SQL Language of the SQLite Engine*



## 🧭 Introduction

- **SQLite** is a lightweight, server-less, self-contained SQL database engine that stores all its data in a single file. 

- It implements most of the SQL-92 standard and is used in everything from web browsers and mobile apps to embedded systems and operating-system utilities.

- Unlike server databases (SQL Server, MySQL, PostgreSQL), SQLite runs **in-process** with the application — there’s no external service to install or manage. 

- You simply open a file, execute SQL statements, and the engine handles data persistence, indexing, and transactions automatically.

- This guide provides a comprehensive overview of **SQLite’s SQL dialect**, its data model, and its unique implementation details. All examples use clean SQL syntax with explanatory commentary below each code block.

## ⚙️ Understanding SQLite Architecture

SQLite is a **file-based relational database**.

When you connect to a database such as `company.db`, SQLite creates (or opens) a single file on disk containing:

* Tables (data)

* Indexes

* Views

* Triggers

* The database schema

There’s no user authentication, no server configuration, and no networking — the database engine runs entirely in your application process.

SQLite supports both **persistent databases** (`.db` files) and **in-memory databases** (`:memory:`) that exist only during program execution.

## 🧱 Basic SQL Structure

The foundation of SQL in SQLite (and all relational systems) is built around four main statement categories:

| Category | Purpose | Examples |
|-----------|----------|-----------|
| **DDL (Data Definition Language)** | Create or alter database objects. | `CREATE TABLE`, `DROP TABLE`, `CREATE INDEX` |
| **DML (Data Manipulation Language)** | Insert, update, or delete data. | `INSERT`, `UPDATE`, `DELETE` |
| **DQL (Data Query Language)** | Retrieve data. | `SELECT`, `WITH`, `JOIN` |
| **DCL/Transaction Control** | Manage changes. | `BEGIN`, `COMMIT`, `ROLLBACK` |

Each section below explains these categories in practical detail.

## Syntax

- SQL is made up of commands. Commands are typically terminated by a semicolon, which marks the end 
of the command.

```sql
                select id, name from foods; 
                insert into foods values (null, 'Whataburger'); 
                delete from foods where id=413; 
```

- Commands, in turn, are composed of a series of tokens. 

- Tokens can be literals, keywords, identifiers, expressions, or special characters. 

- Tokens are separated by white space, such as spaces, tabs, and newlines.

- The semicolon is used by SQLite as a command terminator, signaling the end of the command to be processed. 

- Literals, also called constants, denote explicit values. There are three kinds: string literals, numeric 
literals, and binary literals. 

- String literals are one or more alphanumeric characters surrounded by single quotes.

```sql
                'Jerry' 
                'Newman' 
                'JujyFruit' 
```

- Keywords are words that have a specific meaning in SQL. These include words such as `SELECT`, `UPDATE`, `INSERT`, `CREATE`, `DROP`, and `BEGIN`. 

- Identifiers refer to specific objects within the database, such as tables or indexes. 

- Keywords are reserved words and may not be used as identifiers. 

- SQL is case insensitive with respect to keywords and identifiers. 

## 📚 Creating Tables and Schemas

Tables store data as rows and columns, similar to spreadsheets but strongly typed and indexed.


```
        CREATE [TEMP] TABLE table_name (column_definitions [, constraints]); 

```

In [ ]:
CREATE TABLE Employees 
(
    EmployeeID   INTEGER PRIMARY KEY,
    FirstName    TEXT NOT NULL,
    LastName     TEXT NOT NULL,
    Department   TEXT,
    HireDate     TEXT DEFAULT CURRENT_DATE,
    Salary       REAL
);

**Explanation:**

* `INTEGER PRIMARY KEY` defines a unique identifier. In SQLite, this automatically creates an **alias for the internal rowid** (a 64-bit integer unique to each record).

* `TEXT`, `REAL`, and `INTEGER` are **type affinities**, not rigid types — SQLite stores values dynamically while maintaining data consistency.

* `DEFAULT CURRENT_DATE` automatically inserts the current date on record creation.

### Viewing Tables

In [ ]:
.tables

In [ ]:
.schema Employees

### Temporary Tables

```

        CREATE [TEMP|TEMPORARY] TABLE table_name (column_definitions [, constraints]);

```

- The temp or temporary keyword creates a temporary table. This kind of table will last only as long your session. As soon as you disconnect, it will be destroyed (if you haven’t already destroyed it manually). 

## ALTER Command

- You can change parts of a table with the alter table command. SQLite’s version of alter table can 
either rename a table or add columns. 

- The general form of the command is as follows: 

```

        ALTER TABLE table_name { rename to name | add column column_def } 

```



In [ ]:
ALTER TABLE contacts 
ADD column email TEXT NOT NULL DEFAULT '' COLLATE nocase;

## Querying Data

- The select command incorporates many of the relational operations through a series of clauses in its syntax. 

- Each clause corresponds to specific relational operation. 

In [ ]:
SELECT [distinct] heading 
FROM tables 
WHERE condition
GROUP BY columns 
HAVING condition 
ORDER BY columns 
LIMIT count, offset; 

- Each keyword — `FROM`, `WHERE`, `HAVING`, and so on—is a separate clause. 

- Each clause consists of the keyword followed by arguments 

- The `SELECT` command starts with `FROM`, which takes one or more input relations; combines them into a single composite relation; and passes it through the subsequent chain of operations. 

- All clauses are optional with the exception of `SELECT`. You must always provide at least this clause to make a valid `SELECT` command.

```sql

        SELECT heading FROM tables WHERE condition;

``` 

- The `FROM` clause is a comma-separated list of one or more tables

## ✍️ Inserting Data

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES ('Jane', 'Doe', 'Finance', 85000);

In [ ]:
INSERT INTO Employees (FirstName, LastName, Department, Salary)
VALUES
  ('John', 'Smith', 'HR', 72000),
  ('Alice', 'Brown', 'IT', 95000),
  ('Bob', 'Miller', 'Finance', 78000);

SQLite enforces constraints (`NOT NULL`, `UNIQUE`, `PRIMARY KEY`) automatically.



## 🔍 Querying Data with SELECT

In [ ]:
SELECT FirstName, LastName, Department
FROM Employees
WHERE Department = 'Finance'
ORDER BY LastName;

In [ ]:
SELECT FirstName || ' ' || LastName AS FullName, Salary AS AnnualPay
FROM Employees;

## ⚖️ Filtering Data with WHERE

Use logical and comparison operators to refine results.

| Operator | Meaning | Example |
|-----------|----------|----------|
| `=` | Equal to | `WHERE Department = 'IT'` |
| `<>` or `!=` | Not equal | `WHERE Department <> 'Finance'` |
| `<`, `>`, `<=`, `>=` | Comparison | `WHERE Salary >= 90000` |
| `AND`, `OR`, `NOT` | Combine conditions | `WHERE Department='IT' AND Salary>80000` |

In [ ]:
SELECT * FROM Employees WHERE Department IS NULL;

## 🧠 Understanding SQL Execution Order in SQLite

- SQL queries are not executed in the same order they’re written. 

- SQLite follows a logical processing order that determines how clauses are evaluated.

```

        FROM → WHERE → GROUP BY → HAVING → SELECT → ORDER BY → LIMIT

```

- SQL applies the `WHERE` clause to each row of the relation produced by the `FROM` clause

- The argument of `WHERE` is a logical predicate. A predicate, in the simplest sense, is just an assertion about something.

- The subject in the `WHERE` clause is a row. The row is the logical subject. The `WHERE` clause is the logical 
predicate. 

- Each row in which the proposition evaluates to true is included (or selected) as part of the 
result. 

- Each row in which it is false is excluded. 

## Operators

- Logical operators (`AND`, `OR`, `NOT`, `IN`) are binary operators that operate on truth values or logical 
expressions. 

- They produce a specific truth value depending on their inputs.

- A useful relational operator is `LIKE`. 

- `LIKE` is similar to equals (=) but is used for matching string values against patterns. 

- A percent symbol (`%`) in the pattern matches any sequence of zero or more characters in the string. 

- An underscore (`_`) in the pattern matches any single character in the string. 

## 🔢 Sorting and Limiting Results

- Limit the size and particular range of the result using the `LIMIT` and `OFFSET` keywords. 

- `LIMIT` specifies the maximum number of records to return. 

- `OFFSET` specifies the number of records to skip. 

-  `ORDER BY`. This clause sorts the result by a column or columns before it is returned. 

- This is important in this example is because the rows returned from `SELECT` are never guaranteed to be in any specific order—the SQL standard declares this. 

- The `ORDER BY` clause is essential if you need to count on the result being in any specific order. 

In [ ]:
SELECT * FROM Employees
ORDER BY Salary DESC;

In [ ]:
SELECT * FROM Employees
ORDER BY EmployeeID
LIMIT 5 OFFSET 10;

## 🔗 Joining Tables

In [ ]:
SELECT e.FirstName, e.LastName, d.DepartmentName
FROM Employees AS e
INNER JOIN Departments AS d
ON e.Department = d.DepartmentName;

In [ ]:
SELECT c.CustomerName, o.OrderID
FROM Customers AS c
LEFT JOIN Orders AS o
ON c.CustomerID = o.CustomerID;

## 🧮 Aggregate Functions and Grouping

In [ ]:
SELECT Department, AVG(Salary) AS AvgSalary
FROM Employees
GROUP BY Department
HAVING AVG(Salary) > 80000;

## 🧩 Subqueries

In [ ]:
SELECT FirstName, LastName
FROM Employees
WHERE Department IN (
    SELECT DepartmentName FROM Departments WHERE Location = 'HQ'
);

In [ ]:
SELECT c.CustomerName
FROM Customers AS c
WHERE EXISTS (
    SELECT 1 FROM Orders AS o WHERE o.CustomerID = c.CustomerID
);

## ⚙️ Updating and Deleting Data

In [ ]:
UPDATE Employees
SET Salary = Salary * 1.10
WHERE Department = 'Sales';

In [ ]:
DELETE FROM Employees
WHERE HireDate < '2020-01-01';

## 🧱 Indexes and Keys

In [ ]:
CREATE INDEX idx_employees_department
ON Employees (Department);

In [ ]:
DROP INDEX idx_employees_department;

### Enabling Foreign Keys

In [ ]:
PRAGMA foreign_keys = ON;

## 🧾 Views

In [ ]:
CREATE VIEW vw_HighEarners AS
SELECT FirstName, LastName, Department, Salary
FROM Employees
WHERE Salary > 90000;

In [ ]:
SELECT * FROM vw_HighEarners;

In [ ]:
DROP VIEW vw_HighEarners;

## 🔄 Transactions

In [ ]:
BEGIN TRANSACTION;
UPDATE Accounts SET Balance = Balance - 500 WHERE AccountID = 1;
UPDATE Accounts SET Balance = Balance + 500 WHERE AccountID = 2;
COMMIT;

## 🧩 Triggers

In [ ]:
CREATE TRIGGER trg_UpdateAudit
AFTER UPDATE ON Employees
FOR EACH ROW
BEGIN
    INSERT INTO AuditLog (Action, TableName, RecordID, Timestamp)
    VALUES ('UPDATE', 'Employees', OLD.EmployeeID, datetime('now'));
END;

In [ ]:
DROP TRIGGER trg_UpdateAudit;

## 🧰 Built-In SQLite Functions

| Category | Function | Example | Description |
|-----------|-----------|----------|--------------|
| **String** | `LOWER(text)` | `UPPER(Name)` | Converts to uppercase |
| **Date/Time** | `date('now')` | | Current date |
| **Math** | `ABS(x)` | | Absolute value |
| **Aggregate** | `GROUP_CONCAT(expr, sep)` | | Concatenate group values |

## 🧮 The WITH Clause (CTEs)

In [ ]:
WITH DeptAverage AS (
    SELECT Department, AVG(Salary) AS AvgSalary
    FROM Employees
    GROUP BY Department
)
SELECT e.FirstName, e.LastName, e.Salary, d.AvgSalary
FROM Employees AS e
JOIN DeptAverage AS d ON e.Department = d.Department
WHERE e.Salary > d.AvgSalary;

## 📊 The PRAGMA Command

In [ ]:
PRAGMA foreign_keys = ON;
PRAGMA table_info(Employees);
PRAGMA database_list;
PRAGMA encoding;

## ⚡ Backup, Export, and Import

In [ ]:
.mode csv
.output employees.csv
SELECT * FROM Employees;
.output stdout

In [ ]:
.mode csv
.import employees.csv Employees

## ⚠️ Common Pitfalls and Best Practices

| Issue | Recommendation |
|--------|----------------|
| **Dynamic Typing** | Use `CHECK` constraints for strict validation. |
| **Foreign Keys Off by Default** | Always enable `PRAGMA foreign_keys=ON;`. |
| **Date Handling** | Dates stored as text; use `strftime()`. |
| **Transactions** | Use explicit transactions for performance. |
| **NULLs in Aggregates** | Use `COALESCE()` to replace NULLs. |
| **Case Sensitivity** | Use `COLLATE BINARY` for strict matching. |

## 🧩 Summary

SQLite’s SQL implementation is both **compact and powerful**.

Key takeaways:

* SQLite is **serverless** — everything lives in one file.

* It uses **dynamic typing** and **type affinities**.

* Supports **transactions, joins, triggers, and views**.

* Ideal for local storage and embedded systems.